# Basic Loading of the Library

**Step 1**: Configure your AI service credentials

Use [this notebook](0-AI-settings.ipynb) first, to choose whether to run these notebooks with OpenAI or Azure OpenAI,
and to save your credentials in the configuration file.

In [3]:
// Load some helper functions, e.g. to load values from settings.json
#!import config/Settings.cs 

The Microsoft.AI.Extension can be imported from the following nuget feed:

In [22]:
#r "nuget: Microsoft.Extensions.AI.OpenAI, 9.0.1-preview.1.24570.5"
#r "nuget: Microsoft.Extensions.AI, 9.0.1-preview.1.24570.5"
#r "nuget: Microsoft.Extensions.Caching.Memory, 9.0.0"
#r "nuget: Microsoft.Extensions.Caching.Abstractions, 9.0.0"





Installed Packages Microsoft.Extensions.AI, 9.0.1-preview.1.24570.5 Microsoft.Extensions.AI.OpenAI, 9.0.1-preview.1.24570.5 microsoft.extensions.caching.abstractions, 9.0.0 Microsoft.Extensions.Caching.Memory, 9.0.0

After adding the nuget package, you can instantiate the library, and try to send a prompt:


In [4]:
using Microsoft.Extensions.AI;
using OpenAI;

// Configure AI service credentials used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

IChatClient client =
    new OpenAIClient(apiKey)
        .AsChatClient("gpt-4o-mini");

Console.WriteLine(await client.CompleteAsync("What is AI?"));

Artificial Intelligence (AI) refers to the simulation of human intelligence in machines that are programmed to think and learn like humans. It encompasses a variety of subfields, including machine learning, natural language processing, robotics, computer vision, and more. 

AI systems can analyze data, recognize patterns, make decisions, and improve their performance over time without being explicitly programmed for every task. There are two main types of AI:

1. **Narrow AI (Weak AI)**: This type performs specific tasks and is designed to operate within a limited context. Examples include virtual assistants like Siri or Alexa, recommendation algorithms, and image recognition systems.

2. **General AI (Strong AI)**: This type would possess the ability to perform any intellectual task that a human can do, but as of now, it remains a theoretical concept and has not been achieved.

AI has applications across many industries, including healthcare, finance, automotive, entertainment, and cu

Now, try using conversation history:

In [5]:
Console.WriteLine(await client.CompleteAsync(
[
    new ChatMessage(ChatRole.System, "You are a helpful AI assistant"),
    new ChatMessage(ChatRole.User, "What is AI?"),
]));

Artificial Intelligence (AI) refers to the simulation of human intelligence processes by machines, especially computer systems. AI encompasses a variety of technologies and methods that enable machines to perform tasks that typically require human intelligence. These tasks include:

1. **Learning**: The ability to improve performance based on experience, often achieved through machine learning algorithms.
2. **Reasoning**: The capability to draw conclusions from given information and make decisions.
3. **Problem-Solving**: The ability to find solutions to complex issues or puzzles.
4. **Perception**: The ability to interpret sensory information, such as images, sounds, and other data.
5. **Natural Language Processing (NLP)**: The capability to understand and generate human language, enabling interaction between humans and machines.

AI can be categorized into two main types:

- **Narrow AI**: Also known as weak AI, this type is designed for specific tasks, such as image recognition, sp

Now, try using chat streaming:

In [6]:
await foreach (var update in client.CompleteStreamingAsync("What is AI?"))
{
    Console.Write(update);
}

Artificial Intelligence (AI) refers to the simulation of human intelligence in machines that are programmed to think and learn like humans. It encompasses a variety of technologies and algorithms that enable computers to perform tasks that typically require human-like cognitive functions, such as understanding natural language, recognizing patterns, solving problems, and making decisions.

There are several types of AI, which can be broadly classified into:

1. **Narrow AI (Weak AI)**: This refers to AI systems that are designed and trained for specific tasks, such as voice assistants (like Siri or Alexa), recommendation systems, or image recognition software. Narrow AI can perform designated tasks very well but lacks general intelligence or understanding of tasks outside its specific programming.

2. **General AI (Strong AI)**: This is a hypothetical form of AI that would possess the ability to understand, learn, and apply intelligence in a way that is indistinguishable from human int

Function calling:

In [17]:
using System.ComponentModel;
using Microsoft.Extensions.AI;
using OpenAI;

// Configure AI service credentials used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

IChatClient openaiClient =
    new OpenAIClient(apiKey)
        .AsChatClient("gpt-4o-mini");

var client = new ChatClientBuilder(openaiClient)
    .UseFunctionInvocation()
    .Build();

ChatOptions chatOptions = new()
{
    Tools = [AIFunctionFactory.Create(GetWeather)]
};

await foreach (var message in client.CompleteStreamingAsync("Do I need an umbrella?", chatOptions))
{
    Console.Write(message);
}

[Description("Gets the weather")]
static string GetWeather() => Random.Shared.NextDouble() > 0.5 ? "It's sunny" : "It's raining";

Error: System.IO.FileNotFoundException: Could not load file or assembly 'Microsoft.Extensions.Logging.Abstractions, Version=9.0.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60'. The system cannot find the file specified.
File name: 'Microsoft.Extensions.Logging.Abstractions, Version=9.0.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60'
   at Submission#13.<<Initialize>>d__0.MoveNext()
   at System.Runtime.CompilerServices.AsyncMethodBuilderCore.Start[TStateMachine](TStateMachine& stateMachine)
   at Submission#13.<Initialize>()
   at Submission#13.<Factory>(Object[] submissionArray)
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

Caching:

In [24]:
using Microsoft.Extensions.AI;
using Microsoft.Extensions.Caching.Distributed;
using Microsoft.Extensions.Caching.Memory;
using Microsoft.Extensions.Options;
using OpenAI;

IDistributedCache cache = new MemoryDistributedCache(Options.Create(new MemoryDistributedCacheOptions()));

IChatClient openaiClient =
    new OpenAIClient(apiKey)
        .AsChatClient("gpt-4o-mini");

IChatClient client = new ChatClientBuilder(openaiClient)
    .UseDistributedCache(cache)
    .Build();

for (int i = 0; i < 3; i++)
{
    await foreach (var message in client.CompleteStreamingAsync("In less than 100 words, what is AI?"))
    {
        Console.Write(message);
    }

    Console.WriteLine();
    Console.WriteLine();
}

Error: System.IO.FileNotFoundException: Could not load file or assembly 'Microsoft.Extensions.Caching.Abstractions, Version=9.0.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60'. The system cannot find the file specified.
File name: 'Microsoft.Extensions.Caching.Abstractions, Version=9.0.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60'
   at Submission#17.<<Initialize>>d__0.MoveNext()
   at System.Runtime.CompilerServices.AsyncMethodBuilderCore.Start[TStateMachine](TStateMachine& stateMachine)
   at Submission#17.<Initialize>()
   at Submission#17.<Factory>(Object[] submissionArray)
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)